### P0 WebAssembly Code Generator Tests

#### Original Author: Emil Sekerinski, McMaster University, revised February 2021

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

#### Error: "WASM: no local arrays, records"

In [ ]:
compileString("""
program p
  var a: 1..10 → integer
""", target = 'wat')

#### Error: "WASM: no nested procedures"

In [ ]:
compileString("""
program p
  procedure q()
    write(5)
  q(7)
""", target = 'wat')

#### Error: "WASM: no more than one result parameter"

In [ ]:
compileString("""
procedure q() → (x, y: integer)
  x := y
program p
  writeln()
""", target = 'wat')

#### Error: "WASM: no structured value parameters"

In [ ]:
compileString("""
type A = 1..10 → integer
procedure q(f: A)
  write(5)
program p
  writeln()
""")

#### Error: "WASM: no structured result parameters"

In [ ]:
compileString("""
type A = 1..10 → integer
procedure q() → (f: A)
  write(5)
program p
  writeln()
""")

#### Relational Operators

```
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $q (param $b i32) 
local.get $b
i32.const 0
i32.eq
local.set $b
)
(func $program
(local $x i32)
local.get $x
i32.const 7
i32.gt_s
call $q
)
(memory 1)
(start $program)
)
```

In [ ]:
compileString("""
procedure q(b: boolean)
  b := b = false
program p
  var x: integer
    q(x > 7)
""", 'relop.wat', target = 'wat')

In [ ]:
!wat2wasm relop.wat

#### Input & Output
```
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $program
(local $x i32)
call $read
local.set $x
i32.const 3
local.get $x
i32.mul
local.set $x
local.get $x
call $write
call $writeln
local.get $x
i32.const 5
i32.mul
call $write
)
(memory 1)
(start $program)
)
```

In [ ]:
compileString("""
program p
  var x: integer
    x ← read(); x := 3 × x
    write(x); writeln()
    write(x × 5)
""", 'write.wat', target = 'wat')

In [ ]:
!wat2wasm write.wat

#### Parameter Passing
```
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $x (mut i32) i32.const 0)
(func $q (param $a i32) (param $b i32)
(local $y i32)
local.get $a
local.set $y
local.get $y
call $write
call $writeln
local.get $b
local.set $a
global.get $x
call $write
local.get $a
call $write
call $writeln
local.get $y
local.set $b
local.get $b
call $write
global.get $x
call $write
call $writeln
local.get $a
call $write
local.get $y
call $write
call $writeln
i32.const 12
i32.load
call $write
)
(func $r (param $c i32)
local.get $c
i32.load
global.get $x
i32.const 1
i32.sub
i32.const 4
i32.mul_s
i32.add
global.get $x
i32.store
i32.const 7
local.get $c
i32.load
global.get $x
i32.const 1
i32.sub
i32.const 4
i32.mul_s
i32.add
i32.load
call $q
global.get $x
call $write
)
(func $program
i32.const 5
global.set $x
i32.const 0
call $r
)
(memory 1)
(start $program)
)
```

In [ ]:
compileString("""
type T = 1..10 → integer
var a: T
procedure q(b: integer, c: integer)
    write(b); write(c)
procedure r() → (d: integer)
    a[3] := 9; d := 5
program p
  var x: integer
  a[2] := 7; q(3, a[2]) // writes 3, 7
  x ← r(); write(x); write(a[3]) // writes 5, 9
""")

In [ ]:
compileString("""
type T = 1..10 → integer
var a: T
procedure q(b: integer, c: integer)
    write(b); write(c)
procedure r() → (d: integer)
    a[3] := 9; d := 5
program p
  var x: integer
  a[2] := 7; q(3, a[2]) // writes 3, 7
  x ← r(); write(x); write(a[3]) // writes 5, 9
""", 'params.wat', target = 'wat')

In [ ]:
!wat2wasm params.wat

In [ ]:
runpywasm('params.wasm')

#### Booleans and Conditions

In [ ]:
compileString("""
program p
  const five = 5
  const seven = 7
  const always = true
  const never = false
  var x, y, z: integer
  var b, t, f: boolean
    x := seven; y := 9; z := 11; t := true; f := false
    if true then write(7) else write(9)    // writes 7
    if false then write(7) else write(9)   // writes 9
    if t then write(7) else write(9)       // writes 7
    if f then write(7) else write(9)       // writes 9
    if ¬ t then write(7) else write(9)     // writes 9
    if ¬ f then write(7) else write(9)     // writes 7
    if t or t then write(7) else write(9)  // writes 7
    if t or f then write(7) else write(9)  // writes 7
    if f or t then write(7) else write(9)  // writes 7
    if f or f then write(7) else write(9)  // writes 9
    if t and t then write(7) else write(9) // writes 7
    if t and f then write(7) else write(9) // writes 9
    if f and t then write(7) else write(9) // writes 9
    if f and f then write(7) else write(9) // writes 9
    writeln()
    b := true
    if b then write(3) else write(5) // writes 3
    b := false
    if b then write(3) else write(5) // writes 5
    b := x < y
    if b then write(x) else write(y) // writes 7
    b := (x > y) or t
    if b then write(3) else write(5) // writes 3
    b := (x > y) or f
    if b then write(3) else write(5) // writes 5
    b := (x = y) or (x > y)
    if b then write(3) else write(5) // writes 5
    b := (x = y) or (x < y)
    if b then write(3) else write(5) // writes 3
    b := f and (x ≥ y)
    if b then write(3) else write(5) // writes 5
    writeln()
    while y > 3 do                   // writes 9, 8, 7, 6, 5, 4
      write(y); y := y - 1
    write(y); writeln()              // writes 3
    if ¬(x < y) and t then
      write(x)                       // writes 7
""", 'cond.wat', target = 'wat')

In [ ]:
!wat2wasm cond.wat

In [ ]:
runpywasm('cond.wasm')

#### Constant Folding, Local & Global Variables

```
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $x (mut i32) i32.const 0)
(global $y (mut i32) i32.const 0)
(func $q  
(local $x i32)
i32.const 3
local.set $x
i32.const 1
if
local.get $x
global.set $y
else
i32.const 7
global.set $y
end
global.get $y
call $write
i32.const 0
if
local.get $x
global.set $y
else
i32.const 7
global.set $y
end
global.get $y
call $write
i32.const 1
if
local.get $x
global.set $y
else
i32.const 7
global.set $y
end
global.get $y
call $write
i32.const 0
if
local.get $x
global.set $y
else
i32.const 7
global.set $y
end
global.get $y
call $write
i32.const 0
if
i32.const 5
call $write
else
i32.const 9
call $write
end
)
(func $program
i32.const 7
global.set $x
call $q
global.get $x
call $write
)
(memory 1)
(start $program)
)
```

In [ ]:
compileString("""
const seven = (9 mod 3 + 5 × 3) div 2
type int = integer
var x, y: integer
procedure q()
  const sotrue = true and true
  const sofalse = false and true
  const alsotrue = false or true
  const alsofalse = false or false
  var x: int
    x := 3
    if sotrue then y := x else y := seven
    write(y) // writes 3
    if sofalse then y := x else y := seven
    write(y) // writes 7
    if alsotrue then y := x else y := seven
    write(y) // writes 3
    if alsofalse then y := x else y := seven
    write(y) // writes 7
    if ¬(true or false) then write(5) else write(9)
program p
  x := 7; q(); write(x) // writes 7
""", 'folding.wat', target = 'wat')

In [ ]:
!wat2wasm folding.wat

In [ ]:
runpywasm('folding.wasm')

#### Procedures
```
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $g (mut i32) i32.const 0)
(func $q (param $v i32) 
(local $l i32)
i32.const 9
local.set $l
local.get $l
local.get $v
i32.gt_s
if
local.get $l
call $write
else
global.get $g
call $write
end
)
(func $program
i32.const 5
global.set $g
i32.const 7
call $q
)
(memory 1)
(start $program)
)
```

In [ ]:
compileString("""
var g: integer          // global variable
procedure q(v: integer) // value parameter
  var l: integer        // local variable
    l := 9
    if l > v then write(l)
    else write(g)
program p
  g := 5; q(7)
""", 'proc.wat', target = 'wat')

In [ ]:
!wat2wasm proc.wat

In [ ]:
runpywasm('proc.wasm')

#### Illustrating Lack of Optimization
```
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $program
(local $x i32)
i32.const 5
local.set $x
local.get $x
i32.const 0
i32.add
local.set $x
i32.const 0
local.get $x
i32.add
local.set $x
local.get $x
i32.const 1
i32.mul
local.set $x
i32.const 1
local.get $x
i32.mul
local.set $x
local.get $x
i32.const 3
i32.add
local.set $x
i32.const 3
local.get $x
i32.add
local.set $x
)
(memory 1)
(start $program)
)
```

In [ ]:
compileString("""
program p
  var x: integer
    x := 5
    x := x + 0
    x := 0 + x
    x := x × 1
    x := 1 × x
    x := x + 3
    x := 3 + x
""", 'opt.wat', target = 'wat')

#### Two-dimensional Array
```
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $y (mut i32) i32.const 0)
(global $b (mut i32) i32.const 0)
(func $program
global.get $y
i32.const 3
i32.sub
i32.const 11
i32.mul
i32.const 0
i32.add
i32.const 4
i32.add
i32.const 0
i32.store
global.get $y
i32.const 3
i32.sub
i32.const 11
i32.mul
i32.const 0
i32.add
global.get $y
i32.const 1
i32.add
i32.const 1
i32.sub
i32.const 1
i32.mul
i32.add
i32.load
global.set $b
)
(memory 1)
(start $program)
)
```

In [ ]:
compileString("""
type R = boolean
type S = 1..11 → R
type T = 3..9 → S
var x: T
var y: integer
var b: boolean
program p
  x[y][5] := false
  b := x[y][y + 1]
""", 'twoD.wat', target = 'wat')